# **Helpful Links**


https://github.com/AminHP/gym-anytrading

In [1]:
#  !pip install tensorflow-gpu==1.15.0 tensorflow==1.15.0 stable-baselines3 gym-anytrading gym

In [2]:
# Helpful Links
# https://github.com/AminHP/gym-anytrading
import pandas as pd
import numpy as np
import gym
import gym_anytrading
from gym_anytrading.envs import TradingEnv, ForexEnv, StocksEnv, Actions, Positions 
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'gym_anytrading'

In [ ]:

# env = gym.make('forex-v0', frame_bound=(50, 100), window_size=10)

# observation = env.reset()
# while True:
#     action = env.action_space.sample()
#     observation, reward, done, info = env.step(action)
#     # env.render()
#     if done:
#         print("info:", info)
#         break

# plt.cla()
# env.render_all()
# plt.show()

### DataPreparation

In [ ]:
USDCHF1440_df = pd.read_csv("../input/usdchf-1day/USDCHF1440.csv",sep='\t',header=None)

USDCHF1440_df.sample(10)

In [ ]:
USDCHF1440_df.columns=["Date","Open","High","Low","Close","Volume"]
USDCHF1440_df.head()

In [ ]:
USDCHF1440_df.tail()

In [ ]:
USDCHF1440_df.info()

In [ ]:
#Taking only first 10000 rows to reduce time
# USDCHF1440_df = USDCHF1440_df[:10000]
# USDCHF1440_df.head()

In [ ]:
USDCHF1440_df.Date = pd.to_datetime(USDCHF1440_df.Date)
USDCHF1440_df.Date.head()



In [ ]:
USDCHF1440_df.describe(include="all")

In [ ]:
USDCHF1440_df.set_index("Date",inplace=True)


In [ ]:
USDCHF1440_df.head()

## Custom Environment
Using different window size and the whole data set

# Considering only prices

In [ ]:

def my_process_data(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Close'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['Close', 'Open', 'High', 'Low']].to_numpy()[start:end]
    return prices, signal_features


class MyForexEnv(ForexEnv):
    _process_data = my_process_data


env = MyForexEnv(df=USDCHF1440_df, window_size=12, frame_bound=(12, len(USDCHF1440_df)))
observation = env.reset()
while True:
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
#     env.render()
    if done:
        print("info:", info)
        break
plt.figure(figsize = (15,4))
plt.cla()
env.render_all()
plt.show()

In [ ]:
#Sell=0 Buy=1 from tradeenv.py
trades_df = pd.DataFrame(env.history)
trades_df.head()

In [ ]:
import seaborn as sns
plt.figure(figsize=(15,4))
sns.lineplot(data=trades_df,x=trades_df.index,y="total_reward")
plt.axhline(y=0,color='r')
plt.title("Total reward at every step",fontsize=14)

In [ ]:
plt.figure(figsize=(15,4))
sns.lineplot(data=trades_df,x=trades_df.index,y="total_profit")
plt.title("Total profit at every step",fontsize=14)

In [ ]:
trades_df.position.value_counts()

In [ ]:
plt.figure(figsize=(15,4))
sns.scatterplot(data=trades_df[:50],x=trades_df[:50].index,hue="position",y="position")
plt.grid('both')
plt.title("Psoition of buys and sells for first 100 trades",fontsize=14)

## Add Custom Indicators

Install and Import New Dependencies
https://github.com/peerchemist/finta

In [ ]:
# !pip install finta

## Calculate SMA, MACD RSI and OBV

In [ ]:
from finta import TA

In [ ]:
USDCHF1440_df['SMA'] = TA.SMA(USDCHF1440_df, 10)
USDCHF1440_df['RSI'] = TA.RSI(USDCHF1440_df)
USDCHF1440_df['OBV'] = TA.OBV(USDCHF1440_df)
MACD_df = TA.MACD(USDCHF1440_df)
USDCHF1440_df['MACD'] = MACD_df["MACD"]
USDCHF1440_df['SIGNAL'] = MACD_df["SIGNAL"]
USDCHF1440_df.fillna(0, inplace=True)

In [ ]:
USDCHF1440_df.head()

## Create New Environments

In [ ]:
def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['SMA', 'RSI', 'OBV' ,'MACD', 'SIGNAL']].to_numpy()[start:end]
    return prices, signal_features

In [ ]:
class MyCustomEnv(ForexEnv):
    _process_data = add_signals
    
env2 = MyCustomEnv(df=USDCHF1440_df, window_size=10, frame_bound=(10,len(USDCHF1440_df)-1000))

In [ ]:
#Showng how many action spaces are there
env2.action_space

In [ ]:
#The observation space is a tensor of size (10,5 with low value limit of -inf and high val limit of inf)
env2.observation_space

In [ ]:
#Example observation space this is what the forex robot sees before taking the action
env2.observation_space.sample()

In [ ]:
#The output of signal_features matches the data_frame.head()
env2.signal_features

In [ ]:
USDCHF1440_df.head()

## Build Environment and Train

In [ ]:
# Stable baselines - rl stuff
# from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines.common.vec_env import DummyVecEnv

env_maker = lambda: env2
env = DummyVecEnv([env_maker])

In [ ]:
#Stock standard Algorithm can be found at https://stable-baselines.readthedocs.io/en/master/guide/algos.html
from stable_baselines import A2C
model = A2C('MlpLstmPolicy', env, verbose=1) #MlpLstmPolicy is available only in stable_baselines not in 3
# from stable_baselines3 import A2C
# model = A2C('MlpPolicy', env, verbose=1) 
model.learn(total_timesteps=100000)

## Evaluation

In [ ]:
env = MyCustomEnv(df=USDCHF1440_df, window_size=10, frame_bound=(len(USDCHF1440_df)-1000,len(USDCHF1440_df)))
obs = env.reset()
while True: 
    obs = obs[np.newaxis, ...]
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        print("info", info)
        break

In [ ]:
env.history.keys()

In [ ]:
trade_history = pd.DataFrame(env.history)
trade_history

In [ ]:
trade_history.position.value_counts()

In [ ]:
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize=(15,4))
sns.lineplot(data=trade_history,x=trade_history.index,y="total_reward")
plt.axhline(y=0,color='r')
plt.title("Total reward at every step",fontsize=14)

In [ ]:
plt.figure(figsize=(15,4))
sns.lineplot(data=trade_history,x=trade_history.index,y="total_profit")
plt.axhline(y=1,color='r')
plt.title("Total profit at every step",fontsize=14)

In [ ]:
trade_history["cum_profit"] = np.cumsum(trade_history.total_profit)
plt.figure(figsize=(15,4))
sns.lineplot(data=trade_history,x=trade_history.index,y="cum_profit")
plt.title("Total cummulative profit at every step",fontsize=14)

In [ ]:
#Short is 0 Long is 1
plt.figure(figsize=(15,4))
sns.scatterplot(data=trade_history[:50],x=trade_history[:50].index,hue="position",y="position")
plt.grid('both')

plt.title("Psoition of long and short positions for first 50 trades",fontsize=14)